In [1]:
import os
line_access_token = os.environ.get('LINE_CHANNEL_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_CHANNEL_SECRET')
port = 5000

In [1]:
pip install line-bot-sdk


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from dotenv import load_dotenv
from linebot.configuration import Configuration

# 使用 Configuration 進行初始化
configuration = Configuration(access_token=line_access_token)
line_bot_api = LineBotApi(configuration=configuration)
handler = WebhookHandler(line_secret)

app = Flask(__name__)

# 台灣所有縣市的代碼
cities = {
    "台北": "6300100",
    "新北": "6500100",
    "基隆": "1001701",
    "桃園": "6800100",
    "新竹市": "1001801",
    "新竹縣": "1000401",
    "苗栗": "1000501",
    "台中": "6600100",
    "彰化": "1000701",
    "南投": "1000801",
    "雲林": "1000901",
    "嘉義市": "1002001",
    "嘉義縣": "1001001",
    "台南": "6700100",
    "高雄": "6400100",
    "屏東": "1001301",
    "宜蘭": "1000201",
    "花蓮": "1001501",
    "台東": "1001401",
    "澎湖": "1001601",
    "金門": "0902001",
    "連江": "0900701"
}

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    
    try:
        handler.handle(body, signature)
    except Exception as e:
        print(f"Error: {e}")
        abort(400)
    return 'OK'

# 處理文字訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text.strip()
    
    if user_message == "選擇城市":
        reply_text = (
            "請選擇想去的城市：\n"
            "台北、新北、基隆、桃園、新竹市、新竹縣、苗栗、台中、彰化、南投、雲林、嘉義市、嘉義縣、\n"
            "台南、高雄、屏東、宜蘭、花蓮、台東、澎湖、金門、連江"
        )
    
    elif user_message in cities:
        city_code = cities[user_message]
        weather_url = f"https://www.cwb.gov.tw/V8/C/W/Town/Town.html?TID={city_code}"
        traffic_info = f"https://maps.google.com/?q={user_message}交通"
        
        reply_text = (
            f"{user_message} 的天氣：\n{weather_url}\n\n"
            f"即時路況：\n{traffic_info}"
        )
    
    else:
        reply_text = "請輸入正確的城市名稱或輸入「選擇城市」查看支援的城市。"

    # 回傳訊息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_text)
    )

if __name__ == "__main__":
    app.run(port=8000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/May/2025 08:31:02] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U4af8225b3ff2889a958695372a6ff69a","events":[]}
